In [2]:
import numpy as np
import PySimpleGUI as sg
import random as random
import time as t

In [3]:
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

In [4]:
rgb_to_hex([255,255,255])

'#ffffff'

In [81]:
sunriseDat = np.genfromtxt("rayleigh_corrected_rgb_array", delimiter = ',')

In [6]:
rgb_to_hex(testDat[0,:])

'#5f36ff'

In [7]:
import PySimpleGUI as psg

l1 = psg.Text('Type here', key='-OUT-', font=('Arial Bold', 20), expand_x=True, justification='center')
t1 = psg.Input('60', enable_events=True, key='-INPUT-', font=('Arial Bold', 20), expand_x=True, justification='left')
b1 = psg.Button('Ok', key='-OK-', font=('Arial Bold', 20))
b2 = psg.Button('Exit', font=('Arial Bold', 20))

layout = [[l1], [t1], [b1, b2]]

window = psg.Window('Input Demo', layout, size=(750, 150))

while True:
   event, values = window.read()
   print(event, values)
   if event == '-INPUT-':
      if values['-INPUT-'][-1] not in ('0123456789'):
         psg.popup("Only digits allowed")
         window['-INPUT-'].update(values['-INPUT-'][:-1])
   if event == psg.WIN_CLOSED or event == 'Exit':
      break
        
window.close()

None {'-INPUT-': None}


In [ ]:
PORT_NAME = 'COM7'        # Windows (Change to the COM port your Arduino is connected to)

# Open the serial port
ser = serial.Serial(PORT_NAME, 9600, timeout=1)
ser.flush()

In [85]:
from scipy.constants import h, c, k
from colour_system import *

testDat = np.genfromtxt("test_rgb_array", delimiter = ',')
duration = 60
temp = 5800
cs = cs_hdtv
lam = np.arange(380., 781., 5)


def rgb_to_hex(rgb):
    
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

def get_color_step(color1, color2, t):

    r = int(color1[0] + (color2[0] - color1[0]) * t)
    g = int(color1[1] + (color2[1] - color1[1]) * t)
    b = int(color1[2] + (color2[2] - color1[2]) * t)

    return (r, g, b)

def planck(lam, T):
    """ Returns the spectral radiance of a black body at temperature T.

    Returns the spectral radiance, B(lam, T), in W.sr-1.m-2 of a black body
    at temperature T (in K) at a wavelength lam (in nm), using Planck's law.

    """

    lam_m = lam / 1.e9
    fac = h*c/lam_m/k/T
    B = 2*h*c**2/lam_m**5 / (np.exp(fac) - 1)
    return B

def blackbody_color(T):
    spec = planck(lam, T)
    rgb = cs.spec_to_rgb(spec)
    return np.round(rgb*255)

def smooth_transition(steps, dataName):
    
    duration = 60
    i = 0
    
    if values['-INPUT-'] != '':
        duration = int(values['-INPUT-'])

    print(duration)

    while i < np.shape(dataName)[0]-1 and event != 'Exit':
        print(i)
        if i == 0:
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(testDat[i,:]))
            window['-BACKGROUND-'].update(value = str(testDat[i,0]) + ', ' + str(testDat[i, 1]) + ', ' + str(testDat[i, 2]))
            window.read(timeout = 100*(duration/60))
            for k in random.sample(range(1, 17), 16):
                window.read(timeout=0)
                #print(int(testDat[i, 0]), int(testDat[i, 1]), int(testDat[i, 1]), 255, k)

        start = dataName[i, :]
        end = dataName[i+1, :]

        for j in range(steps+1):
            t = j/steps
            interColor = get_color_step(start, end, t)

            window['-BACKGROUND-'].update(background_color=rgb_to_hex(interColor)) 
            window['-BACKGROUND-'].update(value = str(interColor[0]) + ', ' + str(interColor[1]) + ', ' + str(interColor[2]))
            window.read(timeout = 100*(duration/60))
            for k in random.sample(range(1,17), 16):
                window.read(timeout=0)
                #print(int(interColor[0]), int(interColor[1]), int(interColor[2]), 255, k)

        i += 1

    if i == np.shape(dataName)[0]-1:
        window['-BACKGROUND-'].update(value = 'Done!')


l1 = sg.Text('R, G, B', font =('Arial Bold', 30), size = (20, 5), background_color = 'black', expand_x=True, key='-BACKGROUND-', justification = 'center')
l2 = sg.Text('Input sequence duration (60s default)', font=('Arial Bold', 15), expand_x=True, justification='left')
l3 = sg.Input('', enable_events=True, key='-INPUT-', font=('Arial Bold', 15), expand_x=True, justification='left')
l4 = sg.Button('Sunrise demo sequence', key = '-SUNRISE-', font =('Arial Bold', 15))
l5 = sg.Button('Sunset demo sequence', key = '-SUNSET-', font =('Arial Bold', 15))
l6 = sg.Text('Input blackbody temperature (5800K default)', font=('Arial Bold', 15), expand_x=True, justification='left')
l7 = sg.Input('', enable_events=True, key='-TEMP-', font=('Arial Bold', 15), expand_x=True, justification='left')
l8 = sg.Button('Show blackbody color', key = '-BLACKBODY-', font = ('Arial Bold', 15))
l9 = sg.Exit(font =('Arial Bold', 15))

layout = [[l1], [l2], [l3], [l4], [l5], [l6], [l7], [l8], [l9]]

# Create the window
window = sg.Window('SUNLAMP', layout, finalize=True)

while True:
        
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
        
    if event == '-INPUT-':
        if len(values['-INPUT-']) > 1:
            if values['-INPUT-'][-1] not in ('0123456789'):
                sg.popup("Only digits allowed")
                window['-INPUT-'].update(values['-INPUT-'][:-1])
        
    if event == '-TEMP-':
        if len(values['-TEMP-']) > 1:
            if values['-TEMP-'][-1] not in ('0123456789'):
                sg.popup("Only digits allowed")
                window['-TEMP-'].update(values['-TEMP-'][:-1])
                
    if event == '-SUNRISE-':
        smooth_transition(50, sunriseDat)
        
    if event == '-SUNSET-':
        smooth_transition(50, np.flipud(sunriseDat))
        
    if event == '-BLACKBODY-':
        if values['-TEMP-'] != '':
            temp = int(values['-TEMP-'])
        print(temp)
        tempColor = blackbody_color(temp)
        print(tempColor)
        window['-BACKGROUND-'].update(background_color=rgb_to_hex(tempColor)) 
        window['-BACKGROUND-'].update(value = str(tempColor[0]) + ', ' + str(tempColor[1]) + ', ' + str(tempColor[2]))
        window.read(timeout = 100*(duration/60))
        
        
window.close()

5800
[255. 233. 220.]
1500
[255.  50.   2.]
1000
[255.   9.   0.]
100
[255.   0.  80.]
10
[0. 0. 0.]


/tmp/ipykernel_5906/2537187456.py:33: RuntimeWarning: overflow encountered in exp
  B = 2*h*c**2/lam_m**5 / (np.exp(fac) - 1)


50
[255.   0.  83.]


In [ ]:
steps = 100
i=0

while i < np.shape(testDat)[0]-1:
    print(i)
    if i == 0:
        window['-BACKGROUND-'].update(background_color=rgb_to_hex(testDat[i,:]))  
        window.read(timeout=100)# Change to desired color 
        #for k in random.sample(range(1, 17), 16):
            #print(int(testDat[i, 0]), int(testDat[i, 1]), int(testDat[i, 1]), 255, k)

    start = testDat[i, :]
    end = testDat[i+1, :]

    for j in range(steps+1):
        t = j/steps
        interColor = get_color_step(start, end, t)

        #window['-BACKGROUND-'].update(background_color=rgb_to_hex(interColor)) 

        #window.read(timeout=100)# Change to desired color 
        #for k in random.sample(range(1,17), 16):
            #print(int(interColor[0]), int(interColor[1]), int(interColor[2]), 255, k)
    i += 1


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def get_color_step(color1, color2, t):
    
    r = int(start_color[0] + (end_color[0] - start_color[0]) * t)
    g = int(start_color[1] + (end_color[1] - start_color[1]) * t)
    b = int(start_color[2] + (end_color[2] - start_color[2]) * t)

    return (r, g, b)


start = [255, 232, 165] 
end = [255, 143, 128]     

steps = 50  # How many steps you want in the transition
colors = []

for i in range(steps + 1):
    t = i / steps
    color = interpolate_color(start, end, t)
    colors.append(color)

fig, ax = plt.subplots(figsize=(5, len(colors)))

for i, color in enumerate(colors):
    rect = patches.Rectangle((0, i), 1, 1, linewidth=0, edgecolor='none', facecolor=(color[0]/255, color[1]/255, color[2]/255))
    ax.add_patch(rect)

ax.set_xlim(0, 1)
ax.set_ylim(0, len(colors))
ax.axis("off")

plt.show()